In [1]:
import json
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib import pyplot as plt

import add_path
import toolkit.allen_helpers.stimuli as st
import toolkit.pipeline.signal as ps
from toolkit.pipeline.data_io import SessionDirectory, FILES
from toolkit.pipeline.global_settings import GLOBAL_SETTINGS

pd.set_option('display.max_columns', None)

with open('config.json') as f:
    config = json.load(f)

## Get session and load data

In [2]:
session_id = config['session_id']
ecephys_structure_acronym = config['ecephys_structure_acronym']
print(f"Session ID: {session_id}")

Session ID: 721123822


In [3]:
session_dir = SessionDirectory(session_id, ecephys_structure_acronym, cache_lfp=True)

probe_info = session_dir.load_probe_info()
if not session_dir.has_lfp_data:
    raise ValueError(f"Session {session_id} has no LFP data")

session = session_dir.session

preferred_orientation = session_dir.load_preferred_orientations().sel(
    layer=[GLOBAL_SETTINGS['layer_of_interest']]).values

layer_of_interest = GLOBAL_SETTINGS['layer_of_interest']

c:\Users\chen8\.conda\envs\allen\lib\site-packages\hdmf\spec\namespace.py:590: UserWarning: Ignoring the following cached namespace(s) because another version is already loaded:
core - cached version: 2.2.2, loaded version: 2.7.0
The loaded extension(s) may not be compatible with the cached extension(s) in the file. Please check the extension documentation and ignore this warning if these versions are compatible.
  self.warn_for_ignored_namespaces(ignored_namespaces)


In [4]:
stimulus_presentations = session.stimulus_presentations
session_type = session.session_type

stimulus_names = st.STIMULUS_NAMES[session_type]
drifting_gratings_stimuli = st.STIMULUS_CATEGORIES[session_type]['drifting_gratings'][:1]
natural_movies_stimuli = st.STIMULUS_CATEGORIES[session_type]['natural_movies']

c:\Users\chen8\.conda\envs\allen\lib\site-packages\hdmf\spec\namespace.py:590: UserWarning: Ignoring the following cached namespace(s) because another version is already loaded:
core - cached version: 2.2.2, loaded version: 2.7.0
The loaded extension(s) may not be compatible with the cached extension(s) in the file. Please check the extension documentation and ignore this warning if these versions are compatible.
  self.warn_for_ignored_namespaces(ignored_namespaces)


In [5]:
# Three different combinations of stimuli
combine_stimulus_name = 'natural_movies'
combine_stimulus_name = 'drifting_gratings'
combine_stimulus_name = 'mixed_stimuli'

match combine_stimulus_name:
    case 'natural_movies':
        stimulus_names = natural_movies_stimuli
    case 'drifting_gratings':
        stimulus_names = drifting_gratings_stimuli
    case 'mixed_stimuli':
        stimulus_names = drifting_gratings_stimuli + natural_movies_stimuli

In [ ]:
# Filter conditions
filter_orientation = False  # whether use only preferred orientation

dg_stim = drifting_gratings_stimuli[0]  # first drifting grating stimulus
if dg_stim in stimulus_names:
    stimulus_trials = st.get_stimulus_trials(stimulus_presentations, dg_stim)
    conditions = st.presentation_conditions(stimulus_trials.presentations)

    valid_conditions = ps.filter_conditions(conditions[0])
    if filter_orientation:
        valid_conditions = valid_conditions.sel(orientation=preferred_orientation)
    valid_presentations = np.sort(np.concatenate([conditions[1][c] for c in valid_conditions.values.ravel()]))

### Load LFP power

In [7]:
lfp_power_dss = session_dir.load_stimulus_lfp_power()

In [ ]:
# Update valid presentations given available presentations in LFP data
if dg_stim in lfp_power_dss:
    valid_presentations = np.intersect1d(valid_presentations, lfp_power_dss[dg_stim].presentation_id)
    valid_trials = st.choose_trials(stimulus_trials, valid_presentations)
    lfp_power_dss[dg_stim] = lfp_power_dss[dg_stim].sel(presentation_id=valid_presentations)

### Load units

In [62]:
select_RS = True
select_layer = False

# Load units info from all sessions
units_info = FILES.load('all_units_info')
# Units in current session
units_info = units_info.loc[units_info['session_id'] == session_id]

# Select only Regular Spiking (RS) units
unit_idx = np.ones(len(units_info), dtype=bool)
if select_RS:
    unit_idx = units_info['unit_type'] == 'RS'
if select_layer:
    if select_layer is True:
        select_layer = [layer_of_interest]
    unit_idx = units_info['layer_acronym'].isin(select_layer)

units_id = units_info.index[unit_idx]
print(f"Number of RS units: {len(units_id)}/{len(units_info)}")

Number of RS units: 32/41


## Analyze data

### Get spike counts